In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
#import necessary libraries

import numpy as np
import tensorflow as tf

In [20]:
# read data

with open("/content/drive/MyDrive/New Word Prediction LSTM/dataset.txt", "r", encoding="utf-8") as file:
    data = file.read()

data

'A lot of people ask me stupid fucking questions.\nA lot of people think that what I say on a record or what I talk about on a record, that I actually do in real life or that I believe in it.\nOr if I say that I wanna kill somebody that I\'m actually gonna do it or that I believe in it.\nWell, shit, if you believe that then I\'ll kill you.\nYou know why?\n\'Cause I\'m a criminal\n\nCriminal\nYou\'re goddamn right\nI\'m a criminal\nYeah, I\'m a criminal\n\nMy words are like a dagger with a jagged edge\nThat\'ll stab you in the head\nWhether you\'re a fag or lez\nOr a homosex, hermaph or a trans-a-vest\nPants or dress\nHate fags? The answer\'s "yes"\n\nHomophobic? Nah, you\'re just heterophobic\nStaring at my jeans, watching my genitals bulging\nThat\'s my motherfucking balls,\nYou\'d better let go of \'em\nThey belong in my scrotum,\nYou\'ll never get hold of \'em\n\n"Hey, it\'s me, Versace\nWhoops, somebody shot me!\nAnd I was just checking the mail."\nGet it? Checking the \'male\'?\n\

In [21]:
# tokenize the words

Tokenizer = tf.keras.preprocessing.text.Tokenizer()

Tokenizer.fit_on_texts([data])

Tokenizer.word_index

{'i': 1,
 'you': 2,
 'the': 3,
 'a': 4,
 'to': 5,
 'and': 6,
 'me': 7,
 'it': 8,
 'in': 9,
 "i'm": 10,
 'but': 11,
 'of': 12,
 'my': 13,
 'just': 14,
 'that': 15,
 'on': 16,
 'this': 17,
 "don't": 18,
 'so': 19,
 "'cause": 20,
 'for': 21,
 'with': 22,
 "it's": 23,
 'your': 24,
 'up': 25,
 'all': 26,
 'be': 27,
 'got': 28,
 'like': 29,
 'when': 30,
 'they': 31,
 'we': 32,
 'not': 33,
 'if': 34,
 'is': 35,
 'go': 36,
 'get': 37,
 'no': 38,
 'back': 39,
 'at': 40,
 'was': 41,
 'these': 42,
 'say': 43,
 'shit': 44,
 'know': 45,
 'his': 46,
 'fuck': 47,
 'or': 48,
 'do': 49,
 'kill': 50,
 'he': 51,
 'as': 52,
 'now': 53,
 'what': 54,
 "ain't": 55,
 'sing': 56,
 'let': 57,
 'from': 58,
 'little': 59,
 'see': 60,
 'am': 61,
 'one': 62,
 'out': 63,
 'can': 64,
 'then': 65,
 'why': 66,
 'too': 67,
 "you're": 68,
 'make': 69,
 'how': 70,
 'said': 71,
 'have': 72,
 "i'mma": 73,
 'rap': 74,
 'wanna': 75,
 'think': 76,
 "i'll": 77,
 "can't": 78,
 'want': 79,
 'even': 80,
 'are': 81,
 'better': 82,


In [22]:
# create song-wise training set

train_set = []

for song in data.split("\n"):
    tokenized_song = Tokenizer.texts_to_sequences([song])[0]

    for i in range(1, len(tokenized_song)):
        train_set.append(tokenized_song[: i+1])

In [23]:
# find the size of longest vector

longest_vector_size = max([len(x) for x in train_set])

longest_vector_size

33

In [24]:
# zero-padding the train_set

padded_train_set = tf.keras.preprocessing.sequence.pad_sequences(train_set, maxlen= longest_vector_size, padding= 'pre')

In [25]:
# divide input and output vectors

X = padded_train_set[:, :-1]
y = padded_train_set[:, -1]

In [26]:
# one hot encode the output
y = tf.keras.utils.to_categorical(y, num_classes=(len(Tokenizer.word_index)+1))

y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [27]:
print(f"input shape -> {X.shape} \noutput shape -> {y.shape}")

input shape -> (8882, 32) 
output shape -> (8882, 2023)


In [46]:
# Create LSTM model

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(2023, 100, input_length=32))
model.add(tf.keras.layers.LSTM(100))
model.add(tf.keras.layers.Dense(2023, activation='softmax'))

In [47]:
# Compile the model

model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [48]:
# model summary

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 32, 100)           202300    
                                                                 
 lstm_9 (LSTM)               (None, 100)               80400     
                                                                 
 dense_7 (Dense)             (None, 2023)              204323    
                                                                 
Total params: 487023 (1.86 MB)
Trainable params: 487023 (1.86 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [49]:
# Train the model

model.fit(X, y, epochs=100)

Epoch 1/100
278/278 [==============================] - 16s 50ms/step - loss: 6.5030 - accuracy: 0.0297
Epoch 2/100
278/278 [==============================] - 14s 50ms/step - loss: 6.0655 - accuracy: 0.0298
Epoch 3/100
278/278 [==============================] - 13s 49ms/step - loss: 5.9559 - accuracy: 0.0385
Epoch 4/100
278/278 [==============================] - 13s 48ms/step - loss: 5.8310 - accuracy: 0.0497
Epoch 5/100
278/278 [==============================] - 13s 49ms/step - loss: 5.6753 - accuracy: 0.0538
Epoch 6/100
278/278 [==============================] - 13s 48ms/step - loss: 5.4904 - accuracy: 0.0727
Epoch 7/100
278/278 [==============================] - 14s 50ms/step - loss: 5.2896 - accuracy: 0.0928
Epoch 8/100
278/278 [==============================] - 12s 44ms/step - loss: 5.0803 - accuracy: 0.1139
Epoch 9/100
278/278 [==============================] - 12s 45ms/step - loss: 4.8794 - accuracy: 0.1428
Epoch 10/100
278/278 [==============================] - 13s 48ms/step - l

In [60]:
# Prediction function

def next_words(text, count=5):
    for i in range(count):
        # tokenize
        token_text = Tokenizer.texts_to_sequences([text])[0]
        # padding
        padded_token_text = tf.keras.preprocessing.sequence.pad_sequences([token_text], maxlen=32, padding='pre')
        # predict
        pos = np.argmax(model.predict(padded_token_text, verbose=0))

        for word,index in Tokenizer.word_index.items():
            if index == pos:
                text = text + " " + word
    print(text)

In [72]:
# Test

text = "drugs"

next_words(text)


drugs really got a hold of


In [73]:
text = "Monica Lewinsky"

next_words(text)

Monica Lewinsky feeling on his nutsack shows


In [74]:
text = "Dale Earnhardt"

next_words(text)

Dale Earnhardt of the trailer park the


In [82]:
text = "Angels"

next_words(text)

Angels fight with devils and it'll
